In [1]:
import mlflow
mlflow.autolog()

In [2]:
# @title Importation données
import pandas as pd
ct = 'QueryResults (1).csv'
df = pd.read_csv(ct)
df

,Id,Title,Body,Tags,Score,ViewCount,AnswerCount
0,20485075,How do I define multiple conditions on a join ...,"<p>Ok, lets say I have order and items tables....",<java><jpa><playframework><playframework-2.0><...,1,1984,2
1,20485079,What is the equivalent of a C# gridview in jav...,"<p>I am trying to learn java, I am working on ...",<c#><java><asp.net><swing><gridview>,2,12333,1
2,20485143,"Listview cursor adapter, always get first item...",<p>I have this piece of code for listview usin...,<android><listview><android-listview><simplecu...,1,1075,2
3,20485144,OpenSSL RSA-2048 unencrypted block is longer t...,<p>I am using the OpenSSL library in order to ...,<c><encryption><cryptography><openssl><rsa>,0,527,1
4,20485176,Returning char into main method,<p>The goal of my program is kind of like whee...,<java><eclipse><string><methods><char>,0,1278,3
...,...,...,...,...,...,...,...
49995,22814994,StructureMap use specific instance of type per...,"<p>Is there a way, using StructureMap (the Dep...",<c#><asp.net><dependency-injection><inversion-...,0,369,2
49996,22815009,Add a reference column migration in Rails 4,<p>A user has many uploads. I want to add a co...,<ruby-on-rails><ruby-on-rails-4><foreign-keys>...,368,397557,8
49997,22815010,Hibernate mapping with group by clause?,<p>I have <code>@OneToMany</code> mapping in a...,<java><sql><hibernate><group-by><hibernate-map...,0,1755,1
49998,22815022,java.lang.NoClassDefFoundError: android.suppor...,<p>I have a class that extends <code>android.s...,<java><android><eclipse><android-fragments><an...,0,1441,1


In [3]:
# @title Filtre des tags

df['Tags'] = df['Tags'].str.replace('><', ',').str.replace('<', '').str.replace('>', '')

from collections import Counter
# Fréquence des mots dans 'Tags'
all_tags = df['Tags'].str.split(',').explode().tolist()
tag_counts = Counter(all_tags)

# Trouver les 50 mots les plus fréquents
top_50_tags = [tag for tag, _ in tag_counts.most_common(50)]

# Filtrer les listes de mots pour ne contenir que les 50 mots les plus fréquents
def filter_tags(tags):
    return [tag for tag in tags if tag in top_50_tags]

df['Tags'] = df['Tags'].str.split(',').apply(filter_tags)

# Filtrer les lignes où 'Tags' est vide après le filtrage
df = df[df['Tags'].apply(len) > 0]

df['nb_tags'] = df['Tags'].apply(lambda x: len(x))
df = df[df['nb_tags']==3]

/var/folders/_l/cjjst_c50lx2nkkr280zbp440000gp/T/ipykernel_86913/3543793987.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nb_tags'] = df['Tags'].apply(lambda x: len(x))


In [4]:
df['Tags']

1              [c#, java, asp.net]
16             [php, mysql, linux]
20           [swift, xcode, macos]
23         [javascript, php, html]
26                 [c#, c, string]
                   ...            
49981    [c#, iphone, objective-c]
49985          [php, jquery, html]
49989    [javascript, jquery, css]
49992     [c#, asp.net, angularjs]
49994           [html, css, swift]
Name: Tags, Length: 6644, dtype: object

In [5]:
# @title Nettoyage du texte
import pickle
import nltk
nltk.download('word_tokenize')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

custom_punkt_path = '/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/corpora/punkt'
punkt_path = nltk.data.find(f'{custom_punkt_path}/english.pickle')

with open(punkt_path, 'rb') as file:
    punkt_model = pickle.load(file)
nltk.data.path.append(custom_punkt_path)

with open('/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/corpora/stopwords/english', 'r') as file:
    custom_stopwords = file.read().splitlines()
    
# Nettoyage
def tokenizer_fct(sentence):
    """Division de mots en texte + suppression de certains caractères"""
    sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
    #word_tokens = word_tokenize(sentence_clean, language='english', path_to_punkt=punkt_path)
    word_tokens = punkt_model.tokenize(sentence_clean)
    return word_tokens


stop_w = custom_stopwords + ['[', ']', ',', '.', ':', '?', '(', ')','<','>','~']
def stop_word_filter_fct(list_words):
    """Suppression de mots sans information+ ponctuations"""
    filtered_w = [w for w in list_words if not w in stop_w]
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    return filtered_w2

def lower_start_fct(list_words):
    """Conversion en lettres minuscules et suppression de préfixes indésirables"""
    lw = [w.lower() for w in list_words if (not w.startswith("@")) and (not w.startswith("#")) and (not w.startswith("http"))]
    return lw

def lemma_fct(list_words):
    """lemmatisation"""
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    return lem_w

def transform_bow_fct(desc_text):
    """fonction de transformation"""
    word_tokens = tokenizer_fct(desc_text)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    transf_desc_text = ' '.join(lw)
    return transf_desc_text

# Prétraitement
df['Cleaned_Body'] = df['Body'].apply(transform_bow_fct)


2023/12/18 21:00:28 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
[nltk_data] Error loading word_tokenize: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     self signed certificate in certificate chain
[nltk_data]     (_ssl.c:997)>


In [6]:
# @title Encoding TF - IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_array = tfidf_vectorizer.fit_transform(df['Cleaned_Body']).toarray()

# Liste des termes correspondant aux colonnes de la matrice
feature_names = tfidf_vectorizer.get_feature_names_out()
# Scores de chaque mot
tfidf_df = pd.DataFrame(tfidf_array, columns=feature_names)

# Concaténez les DataFrames sans utiliser de sample
df.reset_index(drop=True, inplace=True)
df2 = pd.concat([df, tfidf_df], axis=1)

Création de modèles LDA avec 50 sujets

In [50]:
import gensim
import pyLDAvis
import pyLDAvis.gensim as gensimvis

from sklearn.decomposition import LatentDirichletAllocation
corpus = df['Cleaned_Body']
X = tfidf_df

tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
terms = tfidf_vectorizer.get_feature_names_out()

# Convertion de la matrice TF-IDF en une représentation sparse de Gensim
corpus_tfidf_gensim = gensim.matutils.Sparse2Corpus(tfidf_matrix.T)

# Création d'un dictionnaire Gensim à partir de TF-IDF
dictionary = gensim.corpora.Dictionary.from_corpus(corpus_tfidf_gensim, id2word=dict((i, s) for i, s in enumerate(terms)))

# Entraînement du modèle LDA avec Gensim et sklearn
num_topics = 50
lda_model_gensim= gensim.models.LdaModel(corpus=corpus_tfidf_gensim, id2word=dictionary, num_topics=num_topics, random_state=42)
lda_model_sklearn = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_model_sklearn.fit(X)

# Obtention des trois sujets les plus probables pour chaque ligne de tfidf_df
top_n = 3
top_topics_gensim = lda_model_gensim.get_document_topics(corpus_tfidf_gensim)
top_topics_sklearn = lda_model_sklearn.transform(X).argsort(axis=1)[:, -top_n:]

# Création d'une colonne 'lda_predict' avec la liste des sujets les plus probables
df['lda_predict_gensim'] = [sorted(topics, key=lambda x: x[1], reverse=True)[:top_n] for topics in top_topics_gensim]
df['lda_predict_gensim'] = df['lda_predict_gensim'].apply(lambda x: [topic[0] for topic in x])

df['lda_predict_sklearn'] = [topics for topics in top_topics_sklearn]

# Enregistrement du modèle Gensim dans MLflow
with mlflow.start_run(run_name='unsupervised_gensim') as run:
    mlflow.sklearn.log_model(
        sk_model=lda_model_gensim,
        artifact_path='lda_gensim',
        registered_model_name="unsupervised_gensim",
    )

# Enregistrement du modèle sklearn dans MLflow
with mlflow.start_run(run_name='unsupervised_sklearn') as run:
    mlflow.sklearn.log_model(
        sk_model=lda_model_sklearn,
        artifact_path='lda_sklearn',
        registered_model_name="unsupervised_sklearn",
    )


2023/12/18 22:16:28 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '9af99e1311e447149680025a954d9e29', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2023/12/18 22:16:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
2023/12/18 22:16:43 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2023/12/18 22:16:43 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023/12/18 22:16:46 WARNING mlflow.utils.autologging_utils: MLflow autologging encounter

Visualisation avec LDAvis

In [46]:
vis_data = gensimvis.prepare(lda_model_gensim, corpus_tfidf_gensim, dictionary)
pyLDAvis.display(vis_data)

In [51]:
df[['Tags','lda_predict_gensim']]

,Tags,lda_predict_gensim
0,"[c#, java, asp.net]","[10, 17, 4]"
1,"[php, mysql, linux]","[10, 28, 48]"
2,"[swift, xcode, macos]","[10, 14, 12]"
3,"[javascript, php, html]","[10, 48, 12]"
4,"[c#, c, string]","[10, 48, 17]"
...,...,...
6639,"[c#, iphone, objective-c]","[10, 14, 45]"
6640,"[php, jquery, html]","[10, 11, 21]"
6641,"[javascript, jquery, css]","[10, 30]"
6642,"[c#, asp.net, angularjs]","[10, 48, 12]"


Retrouver les tags correspondant aux sujets

In [53]:
from collections import defaultdict

# Création d'un dictionnaire pour compter les correspondances
tag_counts = defaultdict(lambda: defaultdict(int))

# Parcours des lignes pour compter les correspondances entre les tags et les numéros
for index, row in df.iterrows():
    tags = row['Tags']
    lda_topics = row['lda_predict_sklearn']

    # Comptage des correspondances entre les tags et les numéros
    for topic in lda_topics:
        for tag in tags:
            tag_counts[topic][tag] += 1

# Convertir le dictionnaire en DataFrame
results = pd.DataFrame(tag_counts).T
results.fillna(0, inplace=True)
results = results.rename_axis('LDA Topic').reset_index()

In [56]:

topic_to_tag = {}
tag_to_topic = {}

# Parcourir les lignes du DataFrame et associer chaque sujet au tag le plus fréquent
for _, row in results.iterrows():
    topic = row['LDA Topic']
    tags_except_lda = row.drop('LDA Topic')
    
    # Vérifier si le tag a déjà été associé à un autre sujet
    if topic not in topic_to_tag and not tags_except_lda.empty:
        # Si le tag n'a pas encore été associé à un sujet
        most_common_tag = tags_except_lda.idxmax()
        
        # Vérifier si le tag a déjà été associé à un autre sujet
        while most_common_tag in tag_to_topic:
            # Si le tag a déjà été associé, trouver le prochain tag le plus fréquent
            tags_except_lda = tags_except_lda.drop(most_common_tag)
            if tags_except_lda.empty:
                # Si tous les tags ont déjà été associés, sortir de la boucle
                break
            most_common_tag = tags_except_lda.idxmax()
        
        # Associer le sujet au tag le plus fréquent
        topic_to_tag[topic] = most_common_tag
        tag_to_topic[most_common_tag] = topic


# Remplacer les numéros dans df['lda_predict'] par les tags
df['lda_predict_sklearn'] = df['lda_predict_sklearn'].apply(lambda topics: [topic_to_tag.get(t, 'Unknown') for t in topics])
df['lda_predict_gensim'] = df['lda_predict_gensim'].apply(lambda topics: [topic_to_tag.get(t, 'Unknown') for t in topics])

In [14]:
import json
json_data = json.dumps(topic_to_tag)
with open("topic_to_tag.json", "w") as file:
    file.write(json_data)
mlflow.log_artifact("topic_to_tag.json")

import os
os.remove("topic_to_tag.json")

In [58]:
df[['Tags','lda_predict_gensim']]

,Tags,lda_predict_gensim
0,"[c#, java, asp.net]","[css, xcode, django]"
1,"[php, mysql, linux]","[css, html, jquery]"
2,"[swift, xcode, macos]","[css, iphone, .net]"
3,"[javascript, php, html]","[css, jquery, .net]"
4,"[c#, c, string]","[css, jquery, xcode]"
...,...,...
6639,"[c#, iphone, objective-c]","[css, iphone, objective-c]"
6640,"[php, jquery, html]","[css, ios, php]"
6641,"[javascript, jquery, css]","[css, c]"
6642,"[c#, asp.net, angularjs]","[css, jquery, .net]"


Scoring

In [60]:
# @title Scoring LDA personnalisé
def gg(predict):
    gg_scores = []
    if predict == 'lda_predict_gensim':
        res = 'gensim'
    else:
        res = 'sklearn'
        
    for index, row in df.iterrows():
        real_tags = set(row['Tags'])
        lda_tags = set(row[predict])

        common_tags = real_tags.intersection(lda_tags)  # Tags communs entre les deux ensembles

        # Ajustement du score en fonction du nombre de prédictions
        if len(row[predict]) == 3:
            gg_scores.append(len(common_tags) / 3)
        elif len(row[predict]) == 2:
            gg_scores.append(len(common_tags) / 2)
        else:
            gg_scores.append(len(common_tags))

    # Calcul de la moyenne des scores "good guess"
    gg = sum(gg_scores) / len(gg_scores)
    print("gg score sur ",res," = ", round(gg, 2) * 100, "%")

gg('lda_predict_gensim')
gg('lda_predict_sklearn')


gg score sur  gensim  =  9.0 %
gg score sur  sklearn  =  25.0 %


In [61]:
# @title Scoring  LDA coherence score et perplexity
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel

# Transformer le corpus TF-IDF en une liste de listes de mots
tokenized_data = [text.split() for text in df['Cleaned_Body']]

# Créer un dictionnaire Gensim à partir de vos données tokenizées
dico = Dictionary(tokenized_data)

# Calculer la cohérence
coherence_model = CoherenceModel(model=lda_model_gensim, texts=tokenized_data, dictionary=dico, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print("Coherence Score =",round(coherence_score,2))

perplexity = lda_model_sklearn.perplexity(tfidf_df)
print("Perplexity= " ,round(perplexity,0))

Coherence Score = 0.68
Perplexity=  2437.0


Tentative d'utiliser un PCA pour réduire la dimension

In [62]:
#Réduction de dimension

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

pca = PCA(n_components=5)
tfidf_pca = pca.fit_transform(tfidf_df)
df_pca = pd.DataFrame(data=tfidf_pca, columns=['PC1', 'PC2','PC3','PC4','PC5'])
#print(df_pca.head())


df_pca_scaled = pd.DataFrame(scaler.fit_transform(df_pca), columns=df_pca.columns)
X = df_pca_scaled
with mlflow.start_run(run_name='unsupervised_PCA') as run:
    lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda_model.fit(X)
    mlflow.sklearn.log_model(
        sk_model=lda_model,
        artifact_path="model",
        registered_model_name="unsupervised",
    )
# Obtenir les trois sujets les plus probables pour chaque ligne
top_n = 3
top_topics = lda_model.transform(X).argsort(axis=1)[:, -top_n:]

# Créer une colonne 'lda_predict' avec la liste des sujets les plus probables
df['lda_predict'] = [topics for topics in top_topics]

gg_scores = []

for index, row in df.iterrows():
    real_tags = set(row['Tags'])
    lda_tags = set(row['lda_predict'])

    common_tags = real_tags.intersection(lda_tags)  # Tags communs entre les deux ensembles

    if len(common_tags) == 3:
        gg_scores.append(1)
    elif len(common_tags) == 2:
        gg_scores.append(2/3)
    elif len(common_tags) == 1:
        gg_scores.append(1/3)
    else:
        gg_scores.append(0)

# Calcul de la moyenne des scores "good guess"
gg = sum(gg_scores) / len(gg_scores)
print("gg = ", round(gg,2)*100," %")

perplexity = lda_model.perplexity(X)
print("Perplexity= " ,round(perplexity,0))

2023/12/18 22:27:29 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '98fb93ac67624ce6afc8c5b49395952f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2023/12/18 22:27:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
2023/12/18 22:27:30 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2023/12/18 22:27:30 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023/12/18 22:27:33 WARNING mlflow.utils.autologging_utils: MLflow autologging encounter

gg =  0.0  %
Perplexity=  76.0
